In [ ]:
#importing all the necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from PIL import Image
import random

from PIL import ImageFile
from keras import layers
import keras
import warnings
import os
import shutil
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import sklearn.model_selection as sklrn
from matplotlib import pyplot as plt

# Setting up the root Directory

In [ ]:
#setting up the root directory
root_path="C:\\Users\\Moavia computer\\Downloads\\Compressed\\Hashem"

# setting up the directories for the train, test and val

In [ ]:
#dataset name and setting up the directory
dataset_folder_name = os.path.join(root_path, 'data')
#setting up the class labels
class_labels = ['cats', 'dogs']
#intiating the empty list for K fold
X = []
Y = []
setting up the paths
train_path = os.path.join(dataset_folder_name, 'train_set')
test_path = os.path.join(dataset_folder_name, 'test_set')

In [ ]:
# this function is to prepare the X List and Y lists so we can use them in K-fold cross validation
def prepare_name_with_labels(folder_name, dataset_type='train_set'):
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = ""
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = class_labels[i]
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [ ]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [ ]:
#As array for the training purposes
X = np.asarray(X)
Y = np.asarray(Y)

In [ ]:
#X contains names of all the images we have for training
print(X)
print(len(X))
#Y contains the crossponding variables
print(Y)
print(len(Y))

In [ ]:
#this is function which takes all the training imagesfrom there folders and add it to the new folder for training and feeding it to the model 
import os
import shutil
from PIL import Image

def extract_images(folder, destination):
    # loop through all the subfolders in the given folder
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            # loop through all the files in the subfolder
            for file in os.listdir(subfolder_path):
                # check if the file is an image
                if file.endswith(".jpg") or file.endswith(".png"):
                    file_path = os.path.join(subfolder_path, file)
                    # open the image and do something with it
                    with Image.open(file_path) as img:
                        # create a new file path for the image in the destination folder
                        new_file_path = os.path.join(destination, file)
                        # copy the image to the destination folder
                        shutil.copy(file_path, new_file_path)

#setting it up for our directories
extract_images("data//train_set", "data//training_set")

In [ ]:
#setting up the hyperparameter 
batch_size = 32
input_size = (256, 256)
NO_OF_EPOCHS = 10
NO_OF_FOLDS = 5

In [ ]:
#this is our model 1
def cnn_model1(input_shape, num_classes):
    model = tf.keras.Sequential()
# Add input layer with input_shape and float32 dtype
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape, dtype='float32'))
    # Add convolutional layer with 32 filters and 3x3 kernel
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    # Add another convolutional layer with 64 filters and 3x3 kernel
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))  
    # Flatten the output of the convolutional layers
    model.add(tf.keras.layers.Flatten())
    # Add fully connected layer with 512 units and ReLU activation
    model.add(tf.keras.layers.Dense(512, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(128, activation='sigmoid'))

    model.add(layers.Dropout(0.5))
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes
    # Add output layer with num_classes units and softmax activation
    model.add(tf.keras.layers.Dense(units, activation=activation))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
input_shape_with_channel=(256,256,3)
num_classes=2
model=cnn_model1(input_shape_with_channel,num_classes)

In [ ]:
#uncomment this if you want to see the details of the model.
#model.summary()

In [ ]:
def train_and_cross_validate (model, x_data, y_data, n_folds=NO_OF_FOLDS, epochs=NO_OF_EPOCHS, batch_size=batch_size):
    # 
    scores = []
    losses=[]
    #  we load images using the generators
    train_datagen = ImageDataGenerator(rescale=1. / 255.,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)    
    validation_datagen = ImageDataGenerator(rescale=1. / 255)
          
    # prepare cross validation 
    kfold = sklrn.KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    FoldsSetNo = 0 
    for train_ix, test_ix in kfold.split(x_data):
        print ('Folds Set # {0}'.format(FoldsSetNo))
        # select rows for train and test
        xx_train, yy_train, xx_test, yy_test = \
            x_data[train_ix], y_data[train_ix], x_data[test_ix], y_data[test_ix]

        # flow training images in batches for the current folds set
        # for training         
        train_generator = train_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_train,'label':yy_train}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='categorical',
            shuffle = False)
        
        # and for validation         
        validation_generator = validation_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_test,'label':yy_test}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='categorical',
            shuffle=False)

        # fit the model
        history = model.fit(train_generator,
                            epochs=epochs,  # The more we train the more our model fits the data
                            batch_size=batch_size,  # Smaller batch sizes = samller steps towards convergence
                            validation_data=validation_generator,
                            verbose=1)
        # store scores
        scores.append({'acc':np.average(history.history['accuracy']),'val_acc':np.average(history.history['val_accuracy'])})
        losses.append({'loss':np.average(history.history['loss']),'val_loss':np.average(history.history['val_loss'])})
        FoldsSetNo +=1
    return scores, losses
print('Starting training and k-fold cross validation ...')
scores = train_and_cross_validate(model, X, Y)

In [ ]:
print(scores)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in scores:
    train.append(s['acc'])
    validation.append(s['val_acc'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train')
plt.plot(validation , color='red', label='validation')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
print(losses)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in losses:
    train.append(s['loss'])
    validation.append(s['val_loss'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train_loss')
plt.plot(validation , color='red', label='validation_loss')
plt.title('loss_curves')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'validation_loss'], loc='upper left')
plt.show()

In [ ]:
#this is our second model
def cnn_model2(input_shape, num_classes):
  model = tf.keras.Sequential()
  # Add input layer with input_shape and float32 dtype
  model.add(tf.keras.layers.InputLayer(input_shape=input_shape, dtype='float32'))
  # Add 5 convolutional layers with 32, 64, 128, 256, and 512 filters
  model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block2
  model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block3  
  model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block4
  model.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block5
  model.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  # Add max pooling layer after each convolutional layer
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  # Flatten the output of the convolutional layers
  model.add(tf.keras.layers.Flatten())
  
  # Add 2 fully connected layers with 512 and 128 units, respectively
  model.add(tf.keras.layers.Dense(512, activation='sigmoid'))
  model.add(tf.keras.layers.Dense(128, activation='sigmoid'))
    
  model.add(layers.Dropout(0.5))
  # Add output layer with num_classes units and softmax activation
  if num_classes == 2:
    activation = "sigmoid"
    units = 1
  else:
    activation = "softmax"
    units = num_classes
    
  model.add(tf.keras.layers.Dense(units, activation=activation))
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model=cnn_model2(input_shape_with_channel,num_classes)

In [ ]:
#uncomment this if you want to see the details of the model.
#model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import sklearn.model_selection as sklrn

def train_and_cross_validate (model, x_data, y_data, n_folds=NO_OF_FOLDS, epochs=NO_OF_EPOCHS, batch_size=batch_size):
    # 
    scores = []
    losses=[]
    #  Loading images through generators ...
    train_datagen = ImageDataGenerator(rescale=1. / 255.,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)    
    validation_datagen = ImageDataGenerator(rescale=1. / 255)
          
    # prepare cross validation
    kfold = sklrn.KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    FoldsSetNo = 0 
    for train_ix, test_ix in kfold.split(x_data):
        print ('Folds Set # {0}'.format(FoldsSetNo))
        # select rows for train and test
        xx_train, yy_train, xx_test, yy_test = \
            x_data[train_ix], y_data[train_ix], x_data[test_ix], y_data[test_ix]

        # flow training images in batches for the current folds set
        # for training         
        train_generator = train_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_train,'label':yy_train}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='categorical',
            shuffle = False)
        
        # and for validation         
        validation_generator = validation_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_test,'label':yy_test}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='categorical',
            shuffle=False)

        # fit the model
        history = model.fit(train_generator,
                            epochs=epochs,  # The more we train the more our model fits the data
                            batch_size=batch_size,  # Smaller batch sizes = samller steps towards convergence
                            validation_data=validation_generator,
                            verbose=1)
        # store scores
        scores.append({'acc':np.average(history.history['accuracy']),'val_acc':np.average(history.history['val_accuracy'])})
        losses.append({'loss':np.average(history.history['loss']),'val_loss':np.average(history.history['val_loss'])})
        FoldsSetNo +=1
    return scores ,losses
print('Starting training and k-fold cross validation ...')
scores = train_and_cross_validate(model, X, Y)

In [ ]:


print(scores)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in scores:
    train.append(s['acc'])
    validation.append(s['val_acc'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train')
plt.plot(validation , color='red', label='validation')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
print(losses)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in losses:
    train.append(s['loss'])
    validation.append(s['val_loss'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train_loss')
plt.plot(validation , color='red', label='validation_loss')
plt.title('loss_curves')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'validation_loss'], loc='upper left')
plt.show()

In [ ]:
#this is our model number 3
def cnn_model3(input_shape, num_classes):
  model = tf.keras.Sequential()
  # Add input layer with input_shape and float32 dtype
  model.add(tf.keras.layers.InputLayer(input_shape=input_shape, dtype='float32'))
  # Add 5 convolutional layers with 32, 64, 128, 256, and 512 filters
  model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block2
  model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block3  
  model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block4
  model.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  #block5
  model.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))
  # Add max pooling layer after each convolutional layer
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  # Flatten the output of the convolutional layers
  model.add(tf.keras.layers.Flatten())
  
  # Add 2 fully connected layers with 512 and 128 units, respectively
  model.add(tf.keras.layers.Dense(512, activation='relu'))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
    
  model.add(layers.Dropout(0.5))
  # Add output layer with num_classes units and softmax activation
  if num_classes == 2:
    activation = "sigmoid"
    units = 1
  else:
    activation = "softmax"
    units = num_classes
    
  model.add(tf.keras.layers.Dense(units, activation=activation))
  
  return model

In [ ]:
model=cnn_model3(input_shape_with_channel,num_classes)

In [ ]:
#model summary  
#model.summary()

In [ ]:
def train_and_cross_validate (model, x_data, y_data, n_folds=NO_OF_FOLDS, epochs=NO_OF_EPOCHS, batch_size=batch_size):
    # 
    scores = []
    losses = []
    #  Loading images through generators ...
    train_datagen = ImageDataGenerator(rescale=1. / 255.,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)    
    validation_datagen = ImageDataGenerator(rescale=1. / 255)
          
    # prepare cross validation
    kfold = sklrn.KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    FoldsSetNo = 0 
    for train_ix, test_ix in kfold.split(x_data):
        print ('Folds Set # {0}'.format(FoldsSetNo))
        # select rows for train and test
        xx_train, yy_train, xx_test, yy_test = \
            x_data[train_ix], y_data[train_ix], x_data[test_ix], y_data[test_ix]

        # flow training images in batches for the current folds set
        # for training         
        train_generator = train_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_train,'label':yy_train}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='binary',
            shuffle = False)
        
        # and for validation         
        validation_generator = validation_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_test,'label':yy_test}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='binary',
            shuffle=False)

        # fit the model
        history = model.fit(train_generator,
                            epochs=epochs,  # The more we train the more our model fits the data
                            batch_size=batch_size,  # Smaller batch sizes = samller steps towards convergence
                            validation_data=validation_generator,
                            verbose=1)
        # store scores
        scores.append({'acc':np.average(history.history['accuracy']),'val_acc':np.average(history.history['val_accuracy'])})
        losses.append({'loss':np.average(history.history['loss']),'val_loss':np.average(history.history['val_loss'])})
        FoldsSetNo +=1
    return scores ,losses
print('Starting training and k-fold cross validation ...')
scores = train_and_cross_validate(model, X, Y)

In [ ]:
from matplotlib import pyplot as plt
# summarize history for accuracy
print(scores)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in scores:
    train.append(s['acc'])
    validation.append(s['val_acc'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train')
plt.plot(validation , color='red', label='validation')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
print(losses)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in losses:
    train.append(s['loss'])
    validation.append(s['val_loss'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train_loss')
plt.plot(validation , color='red', label='validation_loss')
plt.title('loss_curves')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'validation_loss'], loc='upper left')
plt.show()

## this where we import resnet 18 from the keras_resnet.models API 

In [ ]:
from keras_resnet.models import ResNet18
from keras.layers import Input

In [ ]:
# Define the input tensor
inputs = Input(shape=(224,224,3))

# Initialize the ResNet18 model
model = ResNet18(include_top=True, inputs=inputs, classes=1)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
def train_and_cross_validate (model, x_data, y_data, n_folds=NO_OF_FOLDS, epochs=NO_OF_EPOCHS, batch_size=batch_size):
    # 
    scores = []
    
    #  Loading images through generators ...
    train_datagen = ImageDataGenerator(rescale=1. / 255.,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)    
    validation_datagen = ImageDataGenerator(rescale=1. / 255)
          
    # prepare cross validation
    kfold = sklrn.KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    FoldsSetNo = 0 
    for train_ix, test_ix in kfold.split(x_data):
        print ('Folds Set # {0}'.format(FoldsSetNo))
        # select rows for train and test
        xx_train, yy_train, xx_test, yy_test = \
            x_data[train_ix], y_data[train_ix], x_data[test_ix], y_data[test_ix]

        # flow training images in batches for the current folds set
        # for training         
        train_generator = train_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_train,'label':yy_train}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='binary',
            shuffle = False)
        
        # and for validation         
        validation_generator = validation_datagen.flow_from_dataframe(
            dataframe = pd.DataFrame({'id':xx_test,'label':yy_test}), 
            directory="data//training_set", 
            x_col='id',
            y_col='label',
            batch_size=batch_size,
            target_size=input_size,
            class_mode='binary',
            shuffle=False)

        # fit the model
        history = model.fit(train_generator,
                            epochs=epochs,  # The more we train the more our model fits the data
                            batch_size=batch_size,  # Smaller batch sizes = samller steps towards convergence
                            validation_data=validation_generator,
                            verbose=1)
        # store scores
        scores.append({'acc':np.average(history.history['accuracy']),'val_acc':np.average(history.history['val_accuracy'])})
        losses.append({'loss':np.average(history.history['loss']),'val_loss':np.average(history.history['val_loss'])})
        FoldsSetNo +=1
    return scores, losses
print('Starting training and k-fold cross validation ...')
scores = train_and_cross_validate(model, X, Y)

In [ ]:
print(scores)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in scores:
    train.append(s['acc'])
    validation.append(s['val_acc'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train')
plt.plot(validation , color='red', label='validation')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
print(losses)
train = []
validation = []
plt.subplot(1, 1, 1)
for s in losses:
    train.append(s['loss'])
    validation.append(s['val_loss'])
print(train)
print(validation)
plt.plot(train, color='blue', label='train_loss')
plt.plot(validation , color='red', label='validation_loss')
plt.title('loss_curves')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'validation_loss'], loc='upper left')
plt.show()